In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import xgcm

import seaduck as sd
from seaduck.budget import *

In [2]:
import os
os.listdir('/sciserver/filedb01-02')
os.listdir('/sciserver/filedb02-02')
os.listdir('/sciserver/filedb03-02')

['turb', 'cosmo', 'ocean', 'lsst', 'sql_db', 'fannie_mae']

In [3]:
mean = xr.open_mfdataset('/sciserver/filedb0*-02/ocean/poseidon/daily_mean_ecco/zarr/mean*',engine = 'zarr')

snap = xr.open_mfdataset('/sciserver/filedb0*-02/ocean/poseidon/daily_mean_ecco/zarr/snap*',engine = 'zarr')
bc = xr.open_zarr('/sciserver/filedb09-01/ocean/GM_vel.zarr') #bolus correct

grid = xr.open_zarr('~/ECCO_transport')

ds = xr.merge([mean, snap])

them = ds.reset_coords().assign_coords(grid.coords).astype(float)
them

<xarray.Dataset>
Dimensions:     (time: 9497, Zl: 50, face: 13, Y: 90, X: 90, Z: 50, Xp1: 90,
                 Yp1: 90, time_midp: 9496)
Coordinates: (12/32)
    XC          (face, Y, X) float32 dask.array<chunksize=(7, 90, 90), meta=np.ndarray>
    XG          (face, Yp1, Xp1) float32 dask.array<chunksize=(7, 90, 90), meta=np.ndarray>
    YC          (face, Y, X) float32 dask.array<chunksize=(7, 90, 90), meta=np.ndarray>
    YG          (face, Yp1, Xp1) float32 dask.array<chunksize=(7, 90, 90), meta=np.ndarray>
  * time        (time) datetime64[ns] 1992-01-01T18:00:00 ... 2017-12-31T06:0...
  * time_midp   (time_midp) datetime64[ns] 1992-01-02 1992-01-03 ... 2017-12-31
    ...          ...
    maskS       (Z, face, Yp1, X) int8 dask.array<chunksize=(25, 7, 45, 45), meta=np.ndarray>
    maskW       (Z, face, Y, Xp1) int8 dask.array<chunksize=(25, 7, 45, 45), meta=np.ndarray>
    rA          (face, Y, X) float32 dask.array<chunksize=(7, 90, 90), meta=np.ndarray>
    rAs         (face, Yp1, X) float32 dask.array<chunksize=(7, 90, 90), meta=np.ndarray>
    rAw         (face, Y, Xp1) float32 dask.array<chunksize=(7, 90, 90), meta=np.ndarray>
    rAz         (face, Yp1, Xp1) float32 dask.array<chunksize=(7, 90, 90), meta=np.ndarray>
Data variables: (12/35)
    ADVr_SLT    (time, Zl, face, Y, X) float64 dask.array<chunksize=(1, 50, 13, 90, 90), meta=np.ndarray>
    ADVr_TH     (time, Zl, face, Y, X) float64 dask.array<chunksize=(1, 50, 13, 90, 90), meta=np.ndarray>
    ADVx_SLT    (time, Z, face, Y, Xp1) float64 dask.array<chunksize=(1, 50, 13, 90, 90), meta=np.ndarray>
    ADVx_TH     (time, Z, face, Y, Xp1) float64 dask.array<chunksize=(1, 50, 13, 90, 90), meta=np.ndarray>
    ADVy_SLT    (time, Z, face, Yp1, X) float64 dask.array<chunksize=(1, 50, 13, 90, 90), meta=np.ndarray>
    ADVy_TH     (time, Z, face, Yp1, X) float64 dask.array<chunksize=(1, 50, 13, 90, 90), meta=np.ndarray>
    ...          ...
    oceQsw      (time, face, Y, X) float64 dask.array<chunksize=(1, 13, 90, 90), meta=np.ndarray>
    oceSPtnd    (time, Z, face, Y, X) float64 dask.array<chunksize=(1, 50, 13, 90, 90), meta=np.ndarray>
    ETAN_snap   (time_midp, face, Y, X) float64 dask.array<chunksize=(1, 13, 90, 90), meta=np.ndarray>
    SALT_snap   (time_midp, Z, face, Y, X) float64 dask.array<chunksize=(1, 50, 13, 90, 90), meta=np.ndarray>
    SSH_snap    (time_midp, face, Y, X) float64 dask.array<chunksize=(1, 13, 90, 90), meta=np.ndarray>
    THETA_snap  (time_midp, Z, face, Y, X) float64 dask.array<chunksize=(1, 50, 13, 90, 90), meta=np.ndarray>
Attributes: (12/62)
    Conventions:                     CF-1.8, ACDD-1.3
    acknowledgement:                 This research was carried out by the Jet...
    author:                          Ian Fenty and Ou Wang
    cdm_data_type:                   Grid
    comment:                         Fields provided on the curvilinear lat-l...
    coordinates_comment:             Note: the global 'coordinates' attribute...
    ...                              ...
    time_coverage_duration:          P1D
    time_coverage_end:               1992-01-02T00:00:00
    time_coverage_resolution:        P1D
    time_coverage_start:             1992-01-01T12:00:00
    title:                           ECCO Ocean Temperature and Salinity - Da...
    uuid:                            c2efe382-4168-11eb-8956-0cc47a3f480f

In [4]:
them['u'] = them['UVELMASS']+bc['u_gm']
them['v'] = them['VVELMASS']+bc['v_gm']
them['w'] = them['WVELMASS']+bc['w_gm']

In [5]:
tp = sd.Topology(them)

In [6]:
grid = create_ecco_grid(ds)

In [7]:
path = '/sciserver/filedb02-02/ocean/wenrui_temp/heat/'

In [8]:
%%time
# it = 0
lm = 2
rm = 1

dx= np.array(them.dxG)
dy= np.array(them.dyC)
for it in range(9497):
    if it%300==0:
        print(them['time'][it].values)
    ds = them.isel(time = slice(it,it+1))

    w = np.array(ds.w[0])
    s = np.array(ds.THETA[0])
    sz = third_order_upwind_z(s,w)

    
    u = np.array(ds.u[0])
    sx = np.zeros_like(s)
    for face in range(13):
        xbuffer = buffer_x_withface(s,face,lm,rm,tp)
        uu = u[...,face,:,:]
        dxdx = dx[face]
        u_cfl = np.array(u[...,face,:,:]/dx[face])

        sx[:,face] = third_order_DST_x(xbuffer,u_cfl)

    
    v = np.array(ds.v[0])
    sy = np.zeros_like(s)
    for face in range(13):
        u_cfl = np.array(v[...,face,:,:]/dy[face])
        ybuffer = buffer_y_withface(s,face,lm,rm,tp)
        
        sy[:,face] = third_order_DST_y(ybuffer,u_cfl)

    ds['tx'] = xr.DataArray(sx.reshape(1,50,13,90,90),dims = ('time','Z','face','Y','Xp1'))
    ds['ty'] = xr.DataArray(sy.reshape(1,50,13,90,90),dims = ('time','Z','face','Yp1','X'))
    ds['tz'] = xr.DataArray(sz.reshape(1,50,13,90,90),dims = ('time','Zl','face','Y','X'))

    ds_out = ds[['tx','ty','tz']]
    ds_out = ds_out.drop_vars(['XC','YC','CS','Depth','HFacC','HFacS','HFacW','PHrefC',
                      'SN','XG','YG','drF','dxC','dxG','dyC','dyG','maskC',
                      'maskS','maskW','rA','rAs','rAw','rAz'])
    # ds_out

    strtime = str(them['time'][it].values)[:10]
    # ds_out.nbytes
    ds_out.to_netcdf(path+'wall_t/'+strtime+'.nc',mode = 'w')

1992-01-01T18:00:00.000000000
1992-10-27T12:00:00.000000000
1993-08-23T12:00:00.000000000
1994-06-19T12:00:00.000000000
1995-04-15T12:00:00.000000000
1996-12-05T12:00:00.000000000
1997-10-01T12:00:00.000000000
1998-07-28T12:00:00.000000000
1999-05-24T12:00:00.000000000
2000-03-19T12:00:00.000000000
2001-01-13T12:00:00.000000000
2001-11-09T12:00:00.000000000
2002-09-05T12:00:00.000000000
2003-07-02T12:00:00.000000000
2004-04-27T12:00:00.000000000
2005-02-21T12:00:00.000000000
2005-12-18T12:00:00.000000000
2006-10-14T12:00:00.000000000
2007-08-10T12:00:00.000000000
2008-06-05T12:00:00.000000000
2009-04-01T12:00:00.000000000
2010-01-26T12:00:00.000000000
2010-11-22T12:00:00.000000000
2011-09-18T12:00:00.000000000
2012-07-14T12:00:00.000000000
2013-05-10T12:00:00.000000000
2014-03-06T12:00:00.000000000
2014-12-31T12:00:00.000000000
2015-10-27T12:00:00.000000000
2016-08-22T12:00:00.000000000
2017-06-18T12:00:00.000000000
CPU times: user 5h 30min 10s, sys: 2h 27min, total: 7h 57min 10s
Wall 

In [9]:
1

1

In [10]:
1

1

In [11]:
strtime

'2017-12-31'

In [12]:
it

9496

In [13]:
path+'wall_t/'

'/sciserver/filedb02-02/ocean/wenrui_temp/heat/wall_t/'